In [3]:
import numpy as np
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [4]:
# 데이터 로드
article_df = pd.read_csv("article.csv")
len(article_df)

200

In [5]:
# 데이터 전처리 : 데이터 셋 분리
X_train, X_test, y_train, y_test = train_test_split(
article_df.content, article_df.category, test_size=0.1, random_state=1)
len(X_train), len(X_test), len(y_train), len(y_test)

(180, 20, 180, 20)

In [6]:
# 모델 만들기
clf = Pipeline([
    ('vect', TfidfVectorizer()), # tfidf 사용
    ('clf', MultinomialNB(alpha=0.1)), # alpha 값에 따라 결과가 달라짐 (원래는 최적을 찾아야함)
])

In [7]:
# 학습
model = clf.fit(X_train.values.astype("str"), y_train)

In [8]:
# 모델 성능 확인
y_pred = model.predict(X_test)

In [9]:
list(y_test[:5]), list(y_pred[:5])

([100, 100, 100, 101, 101], [100, 100, 100, 100, 101])

In [10]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         100       0.92      1.00      0.96        11
         101       1.00      0.89      0.94         9

    accuracy                           0.95        20
   macro avg       0.96      0.94      0.95        20
weighted avg       0.95      0.95      0.95        20



#### 모델 사용하기

In [14]:
categories = {
    100: "정치",
    101: "경제",
}

In [15]:
contents = [
    "요즘 환율 주가 예측이 불가",
    "광화문 대한애국당 천막, 격렬 저항 속 철거",
]

In [16]:
datas = {
    "content": contents,
    "category_code": model.predict(contents),
}
df = pd.DataFrame(datas)
df

,content,category_code
0,요즘 환율 주가 예측이 불가,101
1,"광화문 대한애국당 천막, 격렬 저항 속 철거",100


In [31]:
# 확률값 보여주기
round(max(model.predict_proba([contents[0]])[0]),2)

0.66

In [32]:
df["category"] = df["category_code"].apply(lambda data: categories[data])
df["proba"] = df["content"].apply(
    lambda data: round(max(model.predict_proba([data])[0]), 2) )
df

,content,category_code,category,proba
0,요즘 환율 주가 예측이 불가,101,경제,0.66
1,"광화문 대한애국당 천막, 격렬 저항 속 철거",100,정치,0.65


In [33]:
# 모델 저장

In [23]:
pickle.dump(model, open("clf.pkl", "wb")) # clf.pkl 이름으로 저장 (write binary)